In [1]:
!pip install pillow
!pip install Flask numpy tensorflow
!pip install fastapi uvicorn python-multipart

In [ ]:
from flask import Flask, render_template, request
import os
from keras.preprocessing import image
import numpy as np
from keras.models import load_model
import base64

app = Flask(__name__, template_folder='template_folder', static_url_path='/static')

model_path = r"D:\Machine Learning\Manoj Sir Practical Assignment\trained_model.h5"
model = load_model(model_path)

class_name_pattern = [
    'Chilli_leaf_curl', 'Chilli_leaf_spot', 'Chilli_whitefly', 'Chilli_yellowish', 'Chillli_healthy',
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight',
    'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot',
    'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]

suggestions = {
    'Chilli_leaf_curl': "Suggestion for Chilli_leaf_curl: Implement preventive measures, such as maintaining proper spacing between plants and using resistant varieties. Consider applying appropriate medicine.",
    'Chilli_leaf_spot': "Suggestion for Chilli_leaf_spot: Implement preventive measures, such as maintaining proper spacing between plants and using resistant varieties. Consider applying appropriate medicine.",
    'Chilli_whitefly': "Suggestion for Chilli_whitefly: Implement preventive measures, such as maintaining proper spacing between plants and using resistant varieties. Consider applying neem oil or insecticidal soap to control whiteflies.",
    'Chilli_yellowish': "Suggestion for Chilli_yellowish: Implement preventive measures, such as maintaining proper spacing between plants and using resistant varieties. Consider applying appropriate medicine.",
    'Chillli_healthy': "Suggestion for Chillli_healthy: The plant looks healthy. Continue regular care.",
    'Pepper__bell___Bacterial_spot': "Suggestion for Pepper__bell___Bacterial_spot: Use disease-resistant varieties, rotate crops, and avoid overhead irrigation to minimize water splashing. Consider applying appropriate medicine.",
    'Pepper__bell___healthy': "Suggestion for Pepper__bell___healthy: The plant looks healthy. Continue regular care.",
    'Potato___Early_blight': "Suggestion for Potato___Early_blight: Practice crop rotation, use certified disease-free seed potatoes, and apply fungicides preventatively in high-risk areas. Consider applying appropriate medicine.",
    'Potato___Late_blight': "Suggestion for Potato___Late_blight: Practice crop rotation, use certified disease-free seed potatoes, and apply fungicides preventatively in high-risk areas. Consider applying appropriate medicine.",
    'Potato___healthy': "Suggestion for Potato___healthy: The plant looks healthy. Continue regular care.",
    'Tomato_Bacterial_spot': "Suggestion for Tomato_Bacterial_spot: Rotate crops, use resistant varieties, and apply fungicides or insecticides preventatively. Practice good sanitation by removing infected plant material. Consider applying appropriate medicine.",
    'Tomato_Early_blight': "Suggestion for Tomato_Early_blight: Rotate crops, use resistant varieties, and apply fungicides or insecticides preventatively. Practice good sanitation by removing infected plant material. Consider applying appropriate medicine.",
    'Tomato_Late_blight': "Suggestion for Tomato_Late_blight: Rotate crops, use resistant varieties, and apply fungicides or insecticides preventatively. Practice good sanitation by removing infected plant material. Consider applying appropriate medicine.",
    'Tomato_Leaf_Mold': "Suggestion for Tomato_Leaf_Mold: Rotate crops and apply fungicides preventatively. Consider applying appropriate medicine.",
    'Tomato_Septoria_leaf_spot': "Suggestion for Tomato_Septoria_leaf_spot: Rotate crops and apply fungicides preventatively. Consider applying appropriate medicine.",
    'Tomato_Spider_mites_Two_spotted_spider_mite': "Suggestion for Tomato_Spider_mites_Two_spotted_spider_mite: Implement preventive measures and consider applying appropriate medicine.",
    'Tomato__Target_Spot': "Suggestion for Tomato__Target_Spot: Rotate crops and apply fungicides preventatively. Consider applying appropriate medicine.",
    'Tomato__Tomato_YellowLeaf__Curl_Virus': "Suggestion for Tomato__Tomato_YellowLeaf__Curl_Virus: Implement preventive measures and consider applying appropriate medicine.",
    'Tomato__Tomato_mosaic_virus': "Suggestion for Tomato__Tomato_mosaic_virus: Implement preventive measures and consider applying appropriate medicine.",
    'Tomato_healthy': "Suggestion for Tomato_healthy: The plant looks healthy. Continue regular care."
}

def generate_suggestion(predicted_class):
    return suggestions.get(predicted_class, "No specific suggestion available.")

def preprocess_image(img_path):
    try:
        img = image.load_img(img_path, target_size=(180, 180))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        return img_array
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        return None

@app.route('/')
def index():
    return render_template('one.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'file' not in request.files:
            return render_template('one.html', message='No file part')

        file = request.files['file']

        if file.filename == '':
            return render_template('one.html', message='No selected file')

        if file:
            # Generate a unique filename to avoid overwriting
            temp_path = f'temp_image_{hash(file.filename)}.jpg'
            file.save(temp_path)
            img_array = preprocess_image(temp_path)

            if img_array is not None:
                predictions = model.predict(img_array)
                predicted_class = np.argmax(predictions)
                predicted_class_name = class_name_pattern[predicted_class]

                # Get suggestion based on predicted class
                suggested_text = generate_suggestion(predicted_class_name)

                # Encode uploaded image to Base64
                with open(temp_path, "rb") as image_file:
                    encoded_uploaded_image = base64.b64encode(image_file.read()).decode('utf-8')

                # Cleanup: Delete the temporary image file
                os.remove(temp_path)

                # Pass suggested_text and encoded_uploaded_image to the template
                return render_template('one.html', message=f'Predicted class: {predicted_class_name}',
                                       suggested_text=suggested_text, encoded_uploaded_image=encoded_uploaded_image)

    except Exception as e:
        return render_template('one.html', message=f'Error: {str(e)}')

if __name__ == '__main__':
    app.run(port=5002)





 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5002
Press CTRL+C to quit
127.0.0.1 - - [06/Feb/2024 10:48:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2024 10:48:22] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 1s 776ms/step


127.0.0.1 - - [06/Feb/2024 10:48:45] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 63ms/step


...

127.0.0.1 - - [06/Feb/2024 11:13:29] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2024 11:13:38] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 205ms/step


127.0.0.1 - - [06/Feb/2024 12:54:30] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2024 12:54:42] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 116ms/step


127.0.0.1 - - [06/Feb/2024 13:07:19] "POST /predict HTTP/1.1" 200 -
